In [17]:
import os
import re
import numpy as np
import pandas as pd
from pprint import pprint
from tqdm import tqdm_notebook as tqdm

In [133]:
path = "./yazio"
regex = re.compile(r"^([\w-]+)-([\w]+\s?[\w]+)\.csv$")
foods = list({re.findall(regex, file)[0][0] for file in next(os.walk(path))[2] if file.endswith(".csv")})
tables = list({re.findall(regex, file)[0][1] for file in next(os.walk(path))[2] if file.endswith(".csv")})

In [142]:
yazio_df = pd.DataFrame([], columns=columns)
yazio_df.index.name="name"
for path_name in tqdm(list(foods)):
    name = path_name.replace("-", " ")
    df = pd.DataFrame([], columns=["per porzione"])
    for table in tables:
        file_path = "{path}/{name}-{table}.csv".format(path=path, name=path_name, table=table)
        if os.path.exists(file_path):
            df = df.append(pd.read_csv(file_path, usecols=["per porzione", table], index_col=table))
    df = df.transpose()
    df.index = [name]
    df.index.name = yazio_df.index.name
    yazio_df = yazio_df.append(df)

In [144]:
yazio_df.to_csv("csv/yazio.csv")